In [ ]:
############ all the libraried required in the code##################

import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cvxopt
from cvxopt import solvers,matrix
import math
from sklearn.cross_validation import KFold
from math import exp,sqrt
import urllib2
import pickle
#from sklearn.preprocessing import normalize

In [ ]:
###########################plotting the data #####################
def plot_data(X,Y):
    x =[]
    y =[]
    for i in range(X.shape[0]):
        x.append(X[i][0])
        y.append(X[i][1])
    plt.plot(x,y,'ro')              
    plt.xlabel("x")
    plt.ylabel("y")
    plt.title("Plot")
    plt.show()

In [ ]:
################# plotting the support vectors##################

def plotting_SVs(X,Y,index):
    x =[]
    y =[]
    x_sv=[]
    y_sv=[]
    for i in range(X.shape[0]):
        if i in index:
            #print "SV"
            x_sv.append(X[i][0])
            y_sv.append(X[i][1])
            continue
        else:
            #print "Non SV"
            x.append(X[i][0])
            y.append(X[i][1])

    plt.plot(x,y,'ro',x_sv,y_sv,'bo')              
    plt.xlabel("x")
    plt.ylabel("y")
    plt.title("Plot")
    plt.show()

In [ ]:
def norm(Z):
    mean = np.sum(Z,axis=0)/len(Z)
    var = []
    for i in range(Z.shape[1]):
        sum1 = 0
        for j in range(Z.shape[0]):
            sum1 += (Z[j,i] - mean[i])**2
        var.append(1. *sum1/len(Z))

    for i in range(Z.shape[1]):
        for j in range(Z.shape[0]):
            Z[j,i] = (Z[j,i] - mean[i])/sqrt(var[i])
    
    
    
            
    return Z

In [ ]:
############# prediction used for linear SVM #################


def prediction_linear(X,W,W0):
    Y_hat = []
    for x in X:
        if (np.dot(np.transpose(W),x) + W0) > 0:
            Y_hat.append(1)
        else:
            Y_hat.append(-1)
    return np.array(Y_hat)

In [ ]:
############### prediction used for guassian SVM ##########################

def prediction_gaussian(X,X_SV,Y_SV,W0,alpha_SV):
    
    Y_hat = []
    
    for x in X:
        
        sum1 = 0
        
        for i in range(len(alpha_SV)):
            
            sum1 += alpha_SV[i]*Y_SV[i]*gaussian_similarity(X_SV[i],x)
        
        if sum1 + W0 > 0:
            
            
            Y_hat.append(1)
        
        else:
            
            Y_hat.append(-1)
            
    return np.array(Y_hat)   

In [ ]:

#################### prediction for ploynomail SVM ##########################

def prediction_poly(X,X_SV,Y_SV,W0,alpha_SV):
    
    Y_hat = []
    
    for x in X:
        
        sum1 = 0
        
        for i in range(len(alpha_SV)):
            
            sum1 += alpha_SV[i]*Y_SV[i]*poly_similarity(X_SV[i],x)
        
        if sum1 + W0 > 0:
            
            #print sum1 + W0
            
                      
            
            Y_hat.append(1)
        
        else:
            
            Y_hat.append(-1)
            
    return np.array(Y_hat)   

In [ ]:

############## function to give the accuracy#######################

def accuracy(Y,Y_hat):
                
    sum1 = sum([1 for i in range(len(Y)) if Y[i] == Y_hat[i]])
    return (1.0*sum1/len(Y))*100

In [ ]:
############################# training for linear SVM #################################


def training_linear(X,Y,e,c):
    
    P = (np.dot(X,np.transpose(X)) * np.dot(Y,np.transpose(Y)))
      
    q = np.zeros(shape=(X.shape[0],1))
    q.fill(-1)
    
    G1 = np.zeros(shape=(X.shape[0],X.shape[0]))
    np.fill_diagonal(G1,-1)
    
    G2 = np.zeros(shape=(X.shape[0],X.shape[0]))
    np.fill_diagonal(G2,1)
    
    G = np.concatenate((G1,G2))
    
    H1 = np.zeros(shape = (X.shape[0],1))
    H2 = np.full((X.shape[0],1),c)
    H = np.concatenate((H1,H2))
    
    A = np.transpose(Y)
    
    b = 0
    
    soln = solvers.qp(matrix(P,tc = 'd'),
                  matrix(q,tc = 'd'),
                  matrix(G,tc = 'd'),
                  matrix(H,tc = 'd'),
                  matrix(A,tc = 'd'),
                  matrix(b,tc='d'))
    
    alpha = (np.ravel(soln['x']))
    alpha = np.reshape(np.array(alpha),(len(alpha),1))
    
          
    W = alpha*Y*X
    W = W.sum(axis = 0)


    W0 = 0
    n = 0
    index = []
    for i in range(X.shape[0]):
        if alpha[i] > e:
            n += 1
            index.append(i)
            W0 += Y[i] - np.dot(np.transpose(W),X[i])
    W0 = 1. * W0/n
    print alpha
    print len(index)
    
    
    
    return (W,W0,index)
    
   
    
        
        

In [ ]:
############################# training for gaussian SVM #################################

def training_gaussian(X,Y,e,c):
    
    P = (gram_matrix_gaussian(X)) * (np.dot(Y,np.transpose(Y)))
    #print P
    q = np.zeros(shape=(X.shape[0],1))
    q.fill(-1)
    
    G1 = np.zeros(shape=(X.shape[0],X.shape[0]))
    np.fill_diagonal(G1,-1)
    
    G2 = np.zeros(shape=(X.shape[0],X.shape[0]))
    np.fill_diagonal(G2,1)
    
    G = np.concatenate((G1,G2))
    
    H1 = np.zeros(shape = (X.shape[0],1))
    H2 = np.full((X.shape[0],1),c)
    H = np.concatenate((H1,H2))
    
    A = np.transpose(Y)
    
    b = 0
    
    soln = solvers.qp(matrix(P,tc = 'd'),
                  matrix(q,tc = 'd'),
                  matrix(G,tc = 'd'),
                  matrix(H,tc = 'd'),
                  matrix(A,tc = 'd'),
                  matrix(b,tc='d'))
    
    alpha = (np.ravel(soln['x']))
    #alpha = np.reshape(np.array(alpha),(len(alpha),1))
    
    #print alpha
     
    alpha_SV = alpha[alpha>e]
    X_SV = X[alpha>e]
    Y_SV = Y[alpha>e]
    index = np.where(alpha>e)
    
    #print alpha_SV
    #print X_SV
    #print Y_SV
    #print index[0]
    
    print len(index[0])
    
    W0 = 0
    for i in range(len(alpha_SV)):
        sum1 = 0
        for j in range(len(alpha_SV)):
            sum1 += alpha_SV[j]*Y_SV[j]*gaussian_similarity(X[j],X[i])
        W0 += Y[i] - sum1
    
    W0 = 1. * W0/len(alpha_SV) 
    
    #print W0
    
    
    
    return (alpha_SV,X_SV,Y_SV,W0,index[0])
    
    
    

In [ ]:
############################# training for polynomail SVM #################################


def training_poly(X,Y,e,c):
    
    P = (gram_matrix_poly(X)) * (np.dot(Y,np.transpose(Y)))
    #print P
    q = np.zeros(shape=(X.shape[0],1))
    q.fill(-1)
    
    G1 = np.zeros(shape=(X.shape[0],X.shape[0]))
    np.fill_diagonal(G1,-1)
    
    G2 = np.zeros(shape=(X.shape[0],X.shape[0]))
    np.fill_diagonal(G2,1)
    
    G = np.concatenate((G1,G2))
    
    H1 = np.zeros(shape = (X.shape[0],1))
    H2 = np.full((X.shape[0],1),c)
    H = np.concatenate((H1,H2))
    
    A = np.transpose(Y)
    
    b = 0
    
    soln = solvers.qp(matrix(P,tc = 'd'),
                  matrix(q,tc = 'd'),
                  matrix(G,tc = 'd'),
                  matrix(H,tc = 'd'),
                  matrix(A,tc = 'd'),
                  matrix(b,tc='d'))
    
    alpha = (np.ravel(soln['x']))
    #alpha = np.reshape(np.array(alpha),(len(alpha),1))
    
    #print alpha
     
    alpha_SV = alpha[alpha>e]
    X_SV = X[alpha>e]
    Y_SV = Y[alpha>e]
    index = np.where(alpha>e)
    
    #print alpha_SV
    #print X_SV
    #print Y_SV
    #print index[0]
    
    
    W0 = 0
    for i in range(len(alpha_SV)):
        sum1 = 0
        for j in range(len(alpha_SV)):
            sum1 += alpha_SV[j]*Y_SV[j]*poly_similarity(X[j],X[i])
        W0 += Y[i] - sum1
    
    W0 = 1. * W0/len(alpha_SV) 
    
    #print W0
    
    print len(index[0])
    
    return (alpha_SV,X_SV,Y_SV,W0,index[0])
    
    
    

In [ ]:
########### gaussain similarity between two vectors#########################

def gaussian_similarity(X,Y,sigma = 3.0):
    #m = np.dot(np.transpose(X-Y),(X-Y))
    return np.exp((-(np.linalg.norm(X-Y)**2))/(2*sigma**2))

In [ ]:
################# gram matrix with gaussain similarity######################

def gram_matrix_gaussian(X):
    m = len(X)
    gram = np.zeros(shape=(m,m))
    for i in range(m):
        x = X[i]
        for j in range(m):        
            y = X[j]
            gram[i][j] += gaussian_similarity(x,y)
    
    return gram

In [ ]:
########### poly similarity between two vectors#########################

def poly_similarity(x,y,q=3):
    return ((np.dot(np.transpose(x),y) + 1) ** 3)

In [ ]:
################# gram matrix with polynomail similarity######################

def gram_matrix_poly(X):
    m = len(X)
    gram = np.zeros(shape=(m,m))
    for i in range(m):
        x = X[i]
        for j in range(m):        
            y = X[j]
            gram[i][j] += poly_similarity(x,y)
    
    return gram

In [ ]:
################ Linear SVM###########
def linear_SVM(X,Y,e,c):
    
    X_norm = norm(X.copy())
    
    #print X
    
    W,W0,index = training_linear(X_norm[:0.75 * len(X)],Y[:0.75 * len(X)],e,c)
    if X.shape[1] == 2:
        plotting_SVs(X[:0.75 * len(X)],Y[:0.75 * len(X)],index)
    Y_hat = prediction_linear(X_norm[0.75 * len(X):],W,W0)
    print accuracy(Y[0.75 * len(X):],Y_hat)

In [ ]:
###### Gausian Kernel SVM ###############
def gaussian_SVM(X,Y,e,c):
    X_norm = norm(X.copy())
    alpha_SV,X_SV,Y_SV,W0,index = training_gaussian(X_norm[:0.75 * len(X)],Y[:0.75 * len(X)],e,c)
    if X.shape[1] == 2:
        plotting_SVs(X[:0.75 * len(X)],Y[:0.75 * len(X)],index)
    Y_hat = prediction_gaussian(X_norm[0.75 * len(X):],X_SV,Y_SV,W0,alpha_SV)
    print accuracy(Y[0.75 * len(X):],Y_hat)

In [ ]:
############ Polynmial kernel SVM######################
def poly_SVM(X,Y,e,c):
    
    X_norm = norm(X.copy())
    alpha_SV,X_SV,Y_SV,W0,index = training_poly(X_norm[:0.75 * len(X)],Y[:0.75 * len(X)],e,c)
    if X.shape[1] == 2:
        plotting_SVs(X[:0.75 * len(X)],Y[:0.75 * len(X)],index)
    Y_hat = prediction_poly(X_norm[0.75 * len(X):],X_SV,Y_SV,W0,alpha_SV)
    return accuracy(Y[0.75 * len(X):],Y_hat)
    #print zip(Y_hat,Y[300:])



In [ ]:
######generating linearly seperable data###########
'''
X = []
Y = []
for i in range(200):
    X.append([random.uniform(0,5),random.uniform(0,10)])
    Y.append(1)
    X.append([random.uniform(6,12),random.uniform(0,10)])
    Y.append(-1)
X = np.array(X)
Y = np.reshape(np.array(Y),(len(Y),1))

'''

In [ ]:
###pickeling the data for linearly separable data####
'''

linear_X = open("data/linear/X.pickle","wb")
pickle.dump(X,linear_X)
linear_X.close()

linear_Y = open("data/linear/Y.pickle","wb")
pickle.dump(Y,linear_Y)
linear_Y.close()

'''

In [ ]:
#####loading the linearly seperable data#####


linear = open("data/linear/X.pickle", "rb")
X = pickle.load(linear)
linear.close()

linear = open("data/linear/Y.pickle", "rb")
Y = pickle.load(linear)
linear.close()

plot_data(X,Y)


In [ ]:

linear_SVM(X,Y,0.00000001,99999)##best accuracy at 0.1####

#gaussian_SVM(X,Y,0.00000001,99999)##best accuracy at 0.00000001
#poly_SVM(X,Y,0.0001,99999)##GIVES THE BEST ACCURACY##


In [ ]:
#######generating Non seperable data set###############
'''
X = []
Y = []
for i in range(200):
    X.append([random.uniform(0,8),random.uniform(0,8)])
    Y.append(1)
    x = random.uniform(0,15)
    if x > 9:
        X.append([x,random.uniform(0,15)])
    else:
        X.append([x,random.uniform(9,15)])
    Y.append(-1)
X = np.array(X)
Y = np.reshape(np.array(Y),(len(Y),1))
plot_data(X,Y)


'''


In [ ]:
'''
non_linear_X = open("data/non-linear/X.pickle","wb")
pickle.dump(X,non_linear_X)
non_linear_X.close()

non_linear_Y = open("data/non-linear/Y.pickle","wb")
pickle.dump(Y,non_linear_Y)
non_linear_Y.close()

'''

In [ ]:
non_linear = open("data/non-linear/X.pickle", "rb")
X = pickle.load(non_linear)
non_linear.close()

non_linear = open("data/non-linear/Y.pickle", "rb")
Y = pickle.load(non_linear)
non_linear.close()

plot_data(X,Y)

In [ ]:
#linear_SVM(X,Y,0.1,1)
#gaussian_SVM(X,Y,0.000001,999999999999999)
#999999999999999
poly_SVM(X,Y,0.000000001,0.1)

In [ ]:
###################### Externel Data set 1 ########################################

url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt"
data = urllib2.urlopen(url)
data = data.read()
data = data.split("\r\n")
random.shuffle(data)

X = []
Y = []
for i in data:
    i = i.split(',')
    Y.append(int(i[len(i)-1]))
    i = map(float,i[0:len(i)-1])
    #print type(i)
    X.append(i)
    
X = np.array(X)
cat = (set(Y))
Y = np.reshape(np.array(Y),(len(Y),1))

Y[Y == 0] = -1


In [ ]:
#gaussian_SVM(X,Y,0.0001,999)
#poly_SVM(X,Y,0.1,i)


In [ ]:
##### External Data set 2 ##################################

url = "http://archive.ics.uci.edu/ml/machine-learning-databases/balance-scale/balance-scale.data"
data = urllib2.urlopen(url)
data = data.read()
data = data.split("\n")[:-1]
random.shuffle(data)

X = []
Y = []
for i in data:
    
    i = i.split(',')
    
    if (i[0] == 'L' or i[0] =='R'):
        
        if (i[0]=='L'):
            Y.append(-1)
        elif (i[0]=='R'):
            Y.append(1)
        i = map(float,i[1:len(i)])



        X.append(i)
X = np.array(X)
Y = np.reshape(np.array(Y),(len(Y),1))


In [ ]:
#gaussian_SVM(X,Y,0.00000001,1000)
#poly_SVM(X,Y,0.2,999999)

In [ ]:
############### when one of the class has substantailly more example###############

####### this data has only 238 1's in 1000 examples, rest are labeled as -1#########

url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt"
data = urllib2.urlopen(url)
data = data.read()
data = data.split("\r\n")

data=data[:1000]

random.shuffle(data)

X = []
Y = []
for i in data:
    i = i.split(',')
    Y.append(int(i[len(i)-1]))
    i = map(float,i[0:len(i)-1])
    #print type(i)
    X.append(i)
    
X = np.array(X)
cat = (set(Y))
Y = np.reshape(np.array(Y),(len(Y),1))
Y[Y == 0] = -1

In [ ]:
#gaussian_SVM(X,Y,0.00000001,1000)
#poly_SVM(X,Y,0.2,999999)